<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Federated_Learning_Example_with_PySyft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install syft

In [ ]:
pip install --upgrade syft

In [ ]:
import syft as sy

# Step 1: Start a domain node
domain = sy.Domain(name="My Secure Domain")

# Step 2: Create data and upload it to the domain
data = sy.Tensor([1, 2, 3, 4, 5])
data_ptr = data.send(domain)  # Send data to the secure domain

print("Data sent to domain:", data_ptr)

# Step 3: Request access to the data
request = data_ptr.request(reason="Perform computations securely.")
print("Data request created:", request)

# Step 4: Perform operations securely (once access is approved)
result = data_ptr + 5  # Example operation
print("Result of secure computation:", result)

In [ ]:
import syft as sy
from syft.node.node import Node

# Initialize a Node (instead of a Domain)
node = Node(name="My Secure Node")

# Create a tensor and share it
tensor = sy.lib.numpy.array([1, 2, 3, 4])
tensor_ptr = tensor.send(node)

print(f"Tensor sent to node: {tensor_ptr}")

In [ ]:
import syft as sy
print(sy.__version__)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import syft as sy

# Create a virtual worker environment
hook = sy.TorchHook(torch)

# Create virtual workers representing different clients
worker1 = sy.VirtualWorker(hook, id="worker1")
worker2 = sy.VirtualWorker(hook, id="worker2")

# Define a simple model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        return self.fc(x)

# Create some dummy data
data_worker1 = torch.randn(500, 10)
labels_worker1 = torch.randint(0, 2, (500, 1)).float()
data_worker2 = torch.randn(500, 10)
labels_worker2 = torch.randint(0, 2, (500, 1)).float()

# Create datasets and loaders for each worker
dataset_worker1 = TensorDataset(data_worker1, labels_worker1)
dataset_worker2 = TensorDataset(data_worker2, labels_worker2)
dataloader_worker1 = DataLoader(dataset_worker1, batch_size=32, shuffle=True)
dataloader_worker2 = DataLoader(dataset_worker2, batch_size=32, shuffle=True)

# Federate the datasets
federated_train_loader = sy.FederatedDataLoader(
    dataset_worker1.federate((worker1, worker2)), batch_size=32
)

# Initialize model and optimizer
model = MyModel()
optimizer = optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.BCEWithLogitsLoss()

# Training loop
for epoch in range(10):  # Example for 10 epochs
    total_loss = 0.0
    for batch_idx, (data, target) in enumerate(federated_train_loader):
        model.send(data.location)  # Send the model to the right worker
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.get().item()
        model.get()  # Get the model back

    avg_loss = total_loss / len(federated_train_loader)
    print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import syft as sy

# Initialize the Syft context
syft_context = sy.CoreContext()

# Create virtual workers representing different clients
worker1 = sy.VirtualMachine(name="worker1")
worker2 = sy.VirtualMachine(name="worker2")

client1 = worker1.get_root_client()
client2 = worker2.get_root_client()

# Define a simple model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        return self.fc(x)

# Create some dummy data
data_worker1 = torch.randn(500, 10)
labels_worker1 = torch.randint(0, 2, (500, 1)).float()
data_worker2 = torch.randn(500, 10)
labels_worker2 = torch.randint(0, 2, (500, 1)).float()

# Create datasets and loaders for each worker
dataset_worker1 = TensorDataset(data_worker1, labels_worker1)
dataset_worker2 = TensorDataset(data_worker2, labels_worker2)

# Send data to virtual workers
remote_dataset_worker1 = sy.lib.python.List(
    [data_worker1.send(client1), labels_worker1.send(client1)]
)
remote_dataset_worker2 = sy.lib.python.List(
    [data_worker2.send(client2), labels_worker2.send(client2)]
)

# Initialize model and optimizer
model = MyModel()
optimizer = optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.BCEWithLogitsLoss()

# Training loop
for epoch in range(10):  # Example for 10 epochs
    total_loss = 0.0
    for client, remote_dataset in [(client1, remote_dataset_worker1), (client2, remote_dataset_worker2)]:
        # Send model to the worker
        model_ptr = model.send(client)

        # Access the remote data
        data_ptr, target_ptr = remote_dataset[0], remote_dataset[1]

        # Perform training on the remote worker
        optimizer.zero_grad()
        output_ptr = model_ptr(data_ptr)
        loss_ptr = loss_fn(output_ptr, target_ptr)
        loss_ptr.backward()
        optimizer.step()

        # Get the loss back
        loss = loss_ptr.get()
        total_loss += loss.item()

        # Get the model back
        model.get()

    avg_loss = total_loss / 2  # Since we have 2 workers
    print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")

In [ ]:
import torch
import syft as sy
import torch.nn as nn
import torch.optim as optim

# Initialize the hook
hook = sy.TorchHook(torch)  # Hook PyTorch to use PySyft

# Create virtual workers
worker1 = sy.VirtualWorker(hook, id="worker1")
worker2 = sy.VirtualWorker(hook, id="worker2")

# Define a simple neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(1, 10)
        self.fc2 = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, optimizer and loss function
model = SimpleNN()
optimizer = optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# Simulate some local training data for each worker
data_worker1 = torch.tensor([[1.0], [2.0], [3.0], [4.0]], requires_grad=True)
target_worker1 = torch.tensor([[1.0], [2.0], [3.0], [4.0]])

data_worker2 = torch.tensor([[5.0], [6.0], [7.0], [8.0]], requires_grad=True)
target_worker2 = torch.tensor([[5.0], [6.0], [7.0], [8.0]])

# Send the data to the workers
data_worker1 = data_worker1.send(worker1)
target_worker1 = target_worker1.send(worker1)

data_worker2 = data_worker2.send(worker2)
target_worker2 = target_worker2.send(worker2)

# Train the model on each worker and perform federated averaging

def train_on_worker(worker, model, data, target, optimizer, loss_fn):
    model.send(worker)
    optimizer.zero_grad()

    prediction = model(data)
    loss = loss_fn(prediction, target)
    loss.backward()
    optimizer.step()

    # Get the updated model back from the worker
    model.get()

# Train the model on each worker
train_on_worker(worker1, model, data_worker1, target_worker1, optimizer, loss_fn)
train_on_worker(worker2, model, data_worker2, target_worker2, optimizer, loss_fn)

# After training, the model parameters are updated on both workers. You can now perform federated averaging.
def federated_avg(models):
    # Averaging the parameters of the models
    model_params = [model.state_dict() for model in models]

    avg_params = {}
    for param in model_params[0]:
        avg_params[param] = torch.stack([model_param[param].float() for model_param in model_params]).mean(0)

    # Set the averaged parameters to the original model
    model.load_state_dict(avg_params)

# Get the models after training on each worker
model_worker1 = model.copy().send(worker1)
model_worker2 = model.copy().send(worker2)

# Federated averaging
federated_avg([model_worker1, model_worker2])

# Now the model has the averaged parameters from both workers

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import syft as sy

# Initialize the hook (This is the updated method for initializing PySyft)
hook = sy.TorchHook(torch)

# Create virtual workers
worker1 = sy.VirtualWorker(hook, id="worker1")
worker2 = sy.VirtualWorker(hook, id="worker2")

# Define a simple neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(1, 10)
        self.fc2 = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, optimizer and loss function
model = SimpleNN()
optimizer = optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# Simulate some local training data for each worker
data_worker1 = torch.tensor([[1.0], [2.0], [3.0], [4.0]], requires_grad=True)
target_worker1 = torch.tensor([[1.0], [2.0], [3.0], [4.0]])

data_worker2 = torch.tensor([[5.0], [6.0], [7.0], [8.0]], requires_grad=True)
target_worker2 = torch.tensor([[5.0], [6.0], [7.0], [8.0]])

# Send the data to the workers
data_worker1 = data_worker1.send(worker1)
target_worker1 = target_worker1.send(worker1)

data_worker2 = data_worker2.send(worker2)
target_worker2 = target_worker2.send(worker2)

# Train the model on each worker and perform federated averaging

def train_on_worker(worker, model, data, target, optimizer, loss_fn):
    model.send(worker)
    optimizer.zero_grad()

    prediction = model(data)
    loss = loss_fn(prediction, target)
    loss.backward()
    optimizer.step()

    # Get the updated model back from the worker
    model.get()

# Train the model on each worker
train_on_worker(worker1, model, data_worker1, target_worker1, optimizer, loss_fn)
train_on_worker(worker2, model, data_worker2, target_worker2, optimizer, loss_fn)

# After training, the model parameters are updated on both workers. You can now perform federated averaging.
def federated_avg(models):
    # Averaging the parameters of the models
    model_params = [model.state_dict() for model in models]

    avg_params = {}
    for param in model_params[0]:
        avg_params[param] = torch.stack([model_param[param].float() for model_param in model_params]).mean(0)

    # Set the averaged parameters to the original model
    model.load_state_dict(avg_params)

# Get the models after training on each worker
model_worker1 = model.copy().send(worker1)
model_worker2 = model.copy().send(worker2)

# Federated averaging
federated_avg([model_worker1, model_worker2])

# Now the model has the averaged parameters from both workers

In [ ]:
import torch
import syft as sy
import torch.nn as nn
import torch.optim as optim

# Initialize the hook
hook = sy.TorchHook(torch)  # Use TorchHook to hook PyTorch to PySyft

# Create virtual workers
worker1 = sy.VirtualWorker(hook, id="worker1")
worker2 = sy.VirtualWorker(hook, id="worker2")

# Define a simple neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(1, 10)
        self.fc2 = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, optimizer and loss function
model = SimpleNN()
optimizer = optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# Simulate some local training data for each worker
data_worker1 = torch.tensor([[1.0], [2.0], [3.0], [4.0]], requires_grad=True)
target_worker1 = torch.tensor([[1.0], [2.0], [3.0], [4.0]])

data_worker2 = torch.tensor([[5.0], [6.0], [7.0], [8.0]], requires_grad=True)
target_worker2 = torch.tensor([[5.0], [6.0], [7.0], [8.0]])

# Send the data to the workers
data_worker1 = data_worker1.send(worker1)
target_worker1 = target_worker1.send(worker1)

data_worker2 = data_worker2.send(worker2)
target_worker2 = target_worker2.send(worker2)

# Train the model on each worker and perform federated averaging

def train_on_worker(worker, model, data, target, optimizer, loss_fn):
    model.send(worker)
    optimizer.zero_grad()

    prediction = model(data)
    loss = loss_fn(prediction, target)
    loss.backward()
    optimizer.step()

    # Get the updated model back from the worker
    model.get()

# Train the model on each worker
train_on_worker(worker1, model, data_worker1, target_worker1, optimizer, loss_fn)
train_on_worker(worker2, model, data_worker2, target_worker2, optimizer, loss_fn)

# After training, the model parameters are updated on both workers. You can now perform federated averaging.
def federated_avg(models):
    # Averaging the parameters of the models
    model_params = [model.state_dict() for model in models]

    avg_params = {}
    for param in model_params[0]:
        avg_params[param] = torch.stack([model_param[param].float() for model_param in model_params]).mean(0)

    # Set the averaged parameters to the original model
    model.load_state_dict(avg_params)

# Get the models after training on each worker
model_worker1 = model.copy().send(worker1)
model_worker2 = model.copy().send(worker2)

# Federated averaging
federated_avg([model_worker1, model_worker2])

# Now the model has the averaged parameters from both workers